In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from IPython.display import clear_output
from dateutil import parser
from datetime import datetime, time
import itertools
from tqdm import tqdm
import sys
import warnings
import datetime
from itertools import permutations 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
# pbar = tqdm(total=36)
# pbar.update(1)
# pbar.close()

In [2]:
def addSecs(Base, Seconds):
    return Base + datetime.timedelta(seconds=Seconds)

def Get_Date_Min_Max(Files_Name, Before, Finish):
    ftr = [3600,60,1]
    
    Files = Files_Name.split("//")[-1].split(".")[0]
    
    Start = Files_Name.split("//")[-1].split(".")[0].split("-")[0].split("_")[-1]
    Start = Start[0:2] + ":" + Start[2:4] + ":" + Start[4:6]
    Start = sum([a*b for a,b in zip(ftr, map(int,Start.split(':')))])

    End =Files_Name.split("//")[-1].split(".")[0].split("-")[1].split("_")[-1]
    End = End[0:2] + ":" + End[2:4] + ":" + End[4:6]
    End = sum([a*b for a,b in zip(ftr, map(int,End.split(':')))])

    Date_1 = Files.split("-")[0].split("_")[:3][::-1]
    Date_2 = Files.split("-")[1].split("_")[:3][::-1]

    Date_1 = datetime.datetime(int(Date_1[0]), int(Date_1[1]), int(Date_1[2]))
    Date_2 = datetime.datetime(int(Date_2[0]), int(Date_2[1]), int(Date_2[2]))

    Start = addSecs(Date_1, Start)
    End = addSecs(Date_2, End)

    if End.hour == 0:
        if Finish == 24:
            Valid_Before = int(24-Before) in range(Start.hour, 24)
        else:
            Valid_Before = int(Finish-Before) in range(Start.hour, 24)
    else:
        Valid_Before = int(Finish-Before) in range(Start.hour, End.hour)

    if End.hour == 0:
        Valid_Finish = int(24) in range(Start.hour, 25)
    else:
        Valid_Finish = int(Finish) in range(Start.hour, End.hour)
    Valid = Valid_Before and Valid_Finish

    Temp = {
            "Valid" : Valid,
            "Date" : Date_1,
            "Start" : Start,
            "End" : End
            }

    return Temp

In [3]:
Patients = ['C0160',
 'C0161',
 'C0163',
 'C0176',
 'C0177',
 'C0178',
 'C0182',
 'C0183',
 'C0186',
 'C0187',
 'C0188',
 'C0189',
 'C0190',
 'C0191',
 'C0192',
 'C0193',
 'C0196',
 'C0197',
 'C0198',
 'C0199',
 'C0200',
 'C0201',
 'C0202',
 'C0203',
 'C0206',
 'C0207',
 'C0209',
 'C0210',
 'C0212',
 'C0213',
 'C0214',
 'C0215',
 'C0216',
 'C0217',
 'C0218',
 'C0219',
 'C0221',
 'C0222',
 'C0223',
 'C0224',
 'C0225',
 'C0228',
 'C0229',
 'C0230',
 'C0231',
 'C0232',
 'C0233',
 'C0235',
 'C0236',
 'C0245',
 'C0248',
 'C0249',
 'C0250',
 'C0251',
 'C0253',
 'C0255',
 'C0256',
 'C0257',
 'C0258',
 'C0259',
 'C0262',
 'C0263',
 'C0264',
 'C0265',
 'C0266',
 'C0267',
 'C0270',
 'C0271',
 'C0272',
 'DCP_00001',
 'DCP_00004',
 'DCP_00005',
 'DCP_00006',
 'DCP_00007',
 'DCP_00008',
 'DCP_00009',
 'DCP_00010',
 'DCP_00011',
 'DCP_00013',
 'DCP_00014',
 'DCP_00015',
 'DCP_00016',
 'DCP_00018',
 'DCP_00019',
 'DCP_00020',
 'DCP_00023',
 'DCP_00024',
 'DCP_00028',
 'DCP_00029',
 'DCP_00030',
 'DCP_00034',
 'DCP_00035',
 'DCP_00036',
 'DCP_00037',
 'DCP_00038',
 'DCP_00039',
 'DCP_00040',
 'DCP_00041',
 'DCP_00042',
 'DCP_00043',
 'DCP_00044',
 'DCP_00045',
 'DCP_00046',
 'DCP_00048',
 'DCP_00049',
 'DCP_00050',
 'DCP_00051',
 'DCP_00053',
 'DCP_00054',
 'DCP_00055',
 'DCP_00056',
 'DCP_00059',
 'DCP_00060',
 'DCP_00061',
 'DCP_00063',
 'DCP_00065',
 'DCP_00066',
 'DCP_00067',
 'DCP_00068',
 'DCP_00070',
 'DCP_00072',
 'DCP_00073',
 'DCP_00074',
 'DCP_00075',
 'DCP_00077',
 'DCP_00084',
 'DCP_00089',
 'DCP_00090',
 'DCP_00092',
 'DCP_00095',
 'DCP_00097',
 'DCP_00100',
 'DCP_00101',
 'DCP_00102',
 'DCP_00106',
 'DCP_00108',
 'DCP_00109',
 'DCP_00110',
 'DCP_00111',
 'DCP_00114',
 'DCP_00118',
 'DCP_00119',
 'DCP_00120',
 'DCP_00121',
 'DCP_00122',
 'DCP_00123',
 'DCP_00124',
 'DCP_00125',
 'DCP_00126',
 'DCP_00133',
 'DCP_00135',
 'DCP_00136',
 'DCP_00140',
 'DCP_00141',
 'DCP_00142']

In [14]:
def Get_ANN_Files(P):
    ANN_Files = []
    for File in os.listdir("..//Patient_Data_Files//" + str(P)):
        if File.endswith(".annauxmix"):
            ANN_Files.append(str(P) + "//"+ str(File))
    return ANN_Files

In [12]:
perm = permutations(range(25), 2) 
Comb = permutations(range(25), 2)
Comb = list(Comb)

# 1,2,4,5,6,8
Good_Comb = []
for C in Comb:
    if C[1]==1 or C[1]==2 or C[1]==4 or C[1]==5 or C[1]==6 or C[1]==8:
        Good_Comb.append(C)

Comb = Good_Comb
Comb[:11]

[(0, 1),
 (0, 2),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 8),
 (1, 2),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 8)]

In [15]:
All_Files = {}
for P in Patients:
    Files = Get_ANN_Files(P)
    All_Files[P] = Files
    del Files

In [16]:
Files_Df_Valid = pd.DataFrame(columns=["Patient_ID", "File", "Before", "Baseline", "Valid"])
Files_Df_Valid

,Patient_ID,File,Before,Baseline,Valid


In [17]:
len(All_Files)

155

In [19]:
All_Files["C0163"]

['C0163//23_11_2018_111900-23_11_2018_145400.annauxmix',
 'C0163//23_11_2018_171800-23_11_2018_221300.annauxmix',
 'C0163//23_11_2018_221430-24_11_2018_000000.annauxmix',
 'C0163//24_11_2018_000000-24_11_2018_011600.annauxmix',
 'C0163//24_11_2018_014500-24_11_2018_071700.annauxmix',
 'C0163//24_11_2018_071730-24_11_2018_103500.annauxmix',
 'C0163//24_11_2018_103930-24_11_2018_141700.annauxmix']

In [22]:
Good_Comb = Comb

In [23]:
Good_Comb.append((1,1))
Good_Comb.append((2,2))
Good_Comb.append((4,4))
Good_Comb.append((5,5))
Good_Comb.append((6,6))
Good_Comb.append((8,8))

In [24]:
Comb = Good_Comb
del Good_Comb

In [25]:
Comb[:11]

[(0, 1),
 (0, 2),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 8),
 (1, 2),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 8)]

In [26]:
pbar = tqdm(total=len(Patients))
for P in Patients:
    Temp_Files = All_Files[P]
    for Time in Comb:
        Baseline = Time[0]
        Bef = Time[1]
        if Bef != 0:
            for F in Temp_Files:
                Files_Df_Valid = Files_Df_Valid.append({ 
                                                            "Patient_ID" : P,
                                                            "File" : F,
                                                            "Before" : Bef,
                                                            "Baseline" : Baseline,
                                                            "Valid" : "",
                                              }, ignore_index=True)
        else:
            pass
    del Temp_Files
    pbar.update(1)
pbar.close()

100%|██████████| 155/155 [14:21<00:00,  4.81s/it]


In [27]:
Files_Df_Valid["Baseline"].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24], dtype=object)

In [28]:
Files_Df_Valid.head()

,Patient_ID,File,Before,Baseline,Valid
0,C0160,C0160//16_11_2018_142000-16_11_2018_142100.ann...,1,0,
1,C0160,C0160//17_11_2018_093200-17_11_2018_093800.ann...,1,0,
2,C0160,C0160//16_11_2018_142000-16_11_2018_142100.ann...,2,0,
3,C0160,C0160//17_11_2018_093200-17_11_2018_093800.ann...,2,0,
4,C0160,C0160//16_11_2018_142000-16_11_2018_142100.ann...,4,0,


In [29]:
pbar = tqdm(total=len(Files_Df_Valid))
for Row in Files_Df_Valid.iterrows():
    F = Row[1][1]
    Bef = Row[1][2]
    Baseline = Row[1][3]
    Valid = Get_Date_Min_Max(F, Bef, Baseline)["Valid"]
    if Valid == True:
        Files_Df_Valid.loc[Row[0], "Valid"] = True
    else:
        Files_Df_Valid.loc[Row[0], "Valid"] = False
    pbar.update(1)
pbar.close()

100%|██████████| 107250/107250 [00:28<00:00, 3710.39it/s]


In [30]:
Files_Df_Valid["Baseline"].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24], dtype=object)

In [31]:
Files_Df_Valid

,Patient_ID,File,Before,Baseline,Valid
0,C0160,C0160//16_11_2018_142000-16_11_2018_142100.ann...,1,0,False
1,C0160,C0160//17_11_2018_093200-17_11_2018_093800.ann...,1,0,False
2,C0160,C0160//16_11_2018_142000-16_11_2018_142100.ann...,2,0,False
3,C0160,C0160//17_11_2018_093200-17_11_2018_093800.ann...,2,0,False
4,C0160,C0160//16_11_2018_142000-16_11_2018_142100.ann...,4,0,False
...,...,...,...,...,...
107245,DCP_00142,DCP_00142//06_02_2020_000000-06_02_2020_083400...,5,5,True
107246,DCP_00142,DCP_00142//05_02_2020_111230-06_02_2020_000000...,6,6,False
107247,DCP_00142,DCP_00142//06_02_2020_000000-06_02_2020_083400...,6,6,True
107248,DCP_00142,DCP_00142//05_02_2020_111230-06_02_2020_000000...,8,8,False


In [34]:
Files_To_Process = Files_Df_Valid.loc[Files_Df_Valid["Valid"] == True]
Files_To_Process.reset_index(drop=True, inplace=True)
Files_To_Process["Index"] = Files_To_Process.index.values.tolist()
Files_To_Process.head()

,Patient_ID,File,Before,Baseline,Valid,Index
0,C0161,C0161//22_11_2018_000000-22_11_2018_092830.ann...,1,2,True,0
1,C0161,C0161//22_11_2018_000000-22_11_2018_092830.ann...,1,3,True,1
2,C0161,C0161//22_11_2018_000000-22_11_2018_092830.ann...,2,3,True,2
3,C0161,C0161//22_11_2018_000000-22_11_2018_092830.ann...,1,4,True,3
4,C0161,C0161//22_11_2018_000000-22_11_2018_092830.ann...,2,4,True,4


In [35]:
Files_To_Process.to_excel("Valids.xlsx", index=False)

In [36]:
Files_To_Process.shape

(14491, 6)

# Testing

- This is only for testing the different componenets of the script

In [21]:
['DCP_00133//18_01_2020_160600-19_01_2020_000000.annauxmix',
  'DCP_00133//19_01_2020_000000-19_01_2020_115130.annauxmix']
Files_Name_One = "DCP_00133//18_01_2020_160600-19_01_2020_000000.annauxmix"
Files_Name_Two = "DCP_00133//19_01_2020_000000-19_01_2020_115130.annauxmix"
Before = 3
Finish = 1

In [ ]:
d1 = datetime.datetime(2020,1,18,) 
d2 = datetime.datetime(2020,1,19) 

In [ ]:
ftr = [3600,60,1]

Files = Files_Name.split("//")[-1].split(".")[0]

Start = Files_Name.split("//")[-1].split(".")[0].split("-")[0].split("_")[-1]
Start = Start[0:2] + ":" + Start[2:4] + ":" + Start[4:6]
Start = sum([a*b for a,b in zip(ftr, map(int,Start.split(':')))])

End =Files_Name.split("//")[-1].split(".")[0].split("-")[1].split("_")[-1]
End = End[0:2] + ":" + End[2:4] + ":" + End[4:6]
End = sum([a*b for a,b in zip(ftr, map(int,End.split(':')))])

Date_1 = Files.split("-")[0].split("_")[:3][::-1]
Date_2 = Files.split("-")[1].split("_")[:3][::-1]

Date_1 = datetime.datetime(int(Date_1[0]), int(Date_1[1]), int(Date_1[2]))
Date_2 = datetime.datetime(int(Date_2[0]), int(Date_2[1]), int(Date_2[2]))

Start = addSecs(Date_1, Start)
End = addSecs(Date_2, End)

if End.hour == 0:
    if Finish == 24:
        Valid_Before = int(24-Before) in range(Start.hour, 24)
    else:
        Valid_Before = int(Finish-Before) in range(Start.hour, 24)
else:
    Valid_Before = int(Finish-Before) in range(Start.hour, End.hour)

if End.hour == 0:
    Valid_Finish = int(24) in range(Start.hour, 25)
else:
    Valid_Finish = int(Finish) in range(Start.hour, End.hour)
Valid = Valid_Before and Valid_Finish

Temp = {
        "Valid" : Valid,
        "Date" : Date_1,
        "Start" : Start,
        "End" : End
        }

Temp